In [156]:
# 지드라이브와 연결
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [158]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

house_df_org = pd.read_csv('/content/gdrive/MyDrive/kaggle/train.csv')
house_test = pd.read_csv('/content/gdrive/MyDrive/kaggle/test.csv')
house_df = house_df_org.copy()
house_df.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [159]:
print('데이터 세트의 Shape:', house_df.shape)
print('\n전체 feature 들의 type \n',house_df.dtypes.value_counts())
isnull_series = house_df.isnull().sum()
print('\nNull 컬럼과 그 건수:\n ', isnull_series[isnull_series > 0].sort_values(ascending=False))

데이터 세트의 Shape: (1460, 81)

전체 feature 들의 type 
 object     43
int64      35
float64     3
dtype: int64

Null 컬럼과 그 건수:
  PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64


In [160]:
house_data = house_df.iloc[:,:-1]

In [161]:
house_target = house_df.iloc[:,-1]

In [162]:
total = pd.concat([house_data, house_test])

In [163]:
house_target = np.log1p(house_target)
house_target

0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

In [164]:
total.reset_index(drop=True, inplace = True)

In [165]:
outliers_c = ['LotFrontage', 'LotArea', 'MasVnrArea',
            'BsmtUnfSF', 'TotalBsmtSF', 'ScreenPorch',
            'MiscVal']

In [166]:
outliers = ['LotFrontage_outlier_index', 'LotArea_outlier_index', 'MasVnrArea_outlier_index',
            'BsmtUnfSF_outlier_index', 'TotalBsmtSF_outlier_index', 'ScreenPorch_outlier_index',
            'MiscVal_outlier_index']

In [167]:
LotFrontage_outlier_index =  [197, 231, 934, 1127, 1298]
LotArea_outlier_index = [249, 313, 335]
MasVnrArea_outlier_index = [58, 70, 115, 297, 691, 808, 1169]
BsmtUnfSF_outlier_index = [224]
TotalBsmtSF_outlier_index = [307, 490, 494, 574, 636, 703, 710, 740, 872, 1100, 1298]
ScreenPorch_outlier_index = [1328, 1386]
MiscVal_outlier_index = [346, 705, 1230, 1457]

In [168]:
for i, j in zip(outliers_c, outliers):
    a = total[i].median()
    print(j)
    for k in eval(j):
        print(k)
        total[i][k] = a

LotFrontage_outlier_index
197
231
934
1127
1298
LotArea_outlier_index
249
313
335
MasVnrArea_outlier_index
58
70
115
297
691
808
1169
BsmtUnfSF_outlier_index
224
TotalBsmtSF_outlier_index
307
490
494
574
636
703
710
740
872
1100
1298
ScreenPorch_outlier_index
1328
1386
MiscVal_outlier_index
346
705
1230
1457


In [169]:
total['LotFrontage'][197] 

68.0

In [170]:
total.drop(['Id','PoolQC' , 'MiscFeature', 'Alley', 'Fence','FireplaceQu'], axis=1 , inplace=True)

In [171]:
total.fillna(house_df.mean(),inplace=True)

In [172]:
null_column_count = total.isnull().sum()[total.isnull().sum() > 0]
print('## Null 피처의 Type :\n', total.dtypes[null_column_count.index])

## Null 피처의 Type :
 MSZoning        object
Utilities       object
Exterior1st     object
Exterior2nd     object
MasVnrType      object
BsmtQual        object
BsmtCond        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
Electrical      object
KitchenQual     object
Functional      object
GarageType      object
GarageFinish    object
GarageQual      object
GarageCond      object
SaleType        object
dtype: object


In [173]:
print('get_dummies() 수행 전 데이터 Shape:', total.shape)
total_ohe = pd.get_dummies(total)
print('get_dummies() 수행 후 데이터 Shape:', total_ohe.shape)

null_column_count = total_ohe.isnull().sum()[total_ohe.isnull().sum() > 0]
print('## Null 피처의 Type :\n', total_ohe.dtypes[null_column_count.index])

get_dummies() 수행 전 데이터 Shape: (2919, 74)
get_dummies() 수행 후 데이터 Shape: (2919, 270)
## Null 피처의 Type :
 Series([], dtype: object)


In [174]:
from scipy.stats import skew

# object가 아닌 숫자형 피처의 칼럼 index 객체 추출.
features_index = total_ohe.dtypes[total_ohe.dtypes != 'object'].index
# total_ohe 칼럼 index를 [ ]로 입력하면 해당하는 칼럼 데이터 세트 반환. apply lambda로 skew( ) 호출
skew_features = total_ohe[features_index].apply(lambda x : skew(x))
# skew(왜곡) 정도가 1 이상인 칼럼만 추출.
skew_features_top = skew_features[skew_features > 1]
print(skew_features_top.sort_values(ascending=False))

Exterior1st_ImStucc    54.000003
RoofMatl_Membran       54.000003
Condition2_RRAn        54.000003
RoofMatl_Metal         54.000003
RoofMatl_ClyTile       54.000003
                         ...    
BsmtFinSF1              1.425228
MSSubClass              1.375457
GrLivArea               1.269358
GarageFinish_Fin        1.177550
GarageType_Detchd       1.054102
Length: 204, dtype: float64


In [175]:
total_ohe[skew_features_top.index] = np.log1p(total_ohe[skew_features_top.index])

In [176]:
total_ohe

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,65.0,9.042040,7,5,2003,2003,5.283204,6.561031,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1,3.044522,80.0,9.169623,6,8,1976,1976,0.000000,6.886532,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2,4.110874,68.0,9.328212,7,5,2001,2002,5.093750,6.188264,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
3,4.262680,60.0,9.164401,7,5,1915,1970,0.000000,5.379897,0.0,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
4,4.110874,84.0,9.565284,8,5,2000,2000,5.860786,6.486161,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,5.081404,21.0,7.568896,4,7,1970,1970,0.000000,0.000000,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2915,5.081404,21.0,7.546974,4,5,1970,1970,0.000000,5.533389,0.0,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
2916,3.044522,160.0,9.903538,5,7,1960,1996,0.000000,7.110696,0.0,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
2917,4.454347,62.0,9.253591,5,5,1992,1992,0.000000,5.823046,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0


In [177]:
total_ohe.isna().sum()[total_ohe.isna().sum() > 0]

Series([], dtype: int64)

In [178]:
total_ohe

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,65.0,9.042040,7,5,2003,2003,5.283204,6.561031,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1,3.044522,80.0,9.169623,6,8,1976,1976,0.000000,6.886532,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2,4.110874,68.0,9.328212,7,5,2001,2002,5.093750,6.188264,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
3,4.262680,60.0,9.164401,7,5,1915,1970,0.000000,5.379897,0.0,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
4,4.110874,84.0,9.565284,8,5,2000,2000,5.860786,6.486161,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,5.081404,21.0,7.568896,4,7,1970,1970,0.000000,0.000000,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2915,5.081404,21.0,7.546974,4,5,1970,1970,0.000000,5.533389,0.0,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
2916,3.044522,160.0,9.903538,5,7,1960,1996,0.000000,7.110696,0.0,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
2917,4.454347,62.0,9.253591,5,5,1992,1992,0.000000,5.823046,0.0,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0


In [179]:
total_ohe.iloc[:1460,:]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,65.0,9.042040,7,5,2003,2003,5.283204,6.561031,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1,3.044522,80.0,9.169623,6,8,1976,1976,0.000000,6.886532,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2,4.110874,68.0,9.328212,7,5,2001,2002,5.093750,6.188264,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
3,4.262680,60.0,9.164401,7,5,1915,1970,0.000000,5.379897,0.000000,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
4,4.110874,84.0,9.565284,8,5,2000,2000,5.860786,6.486161,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,4.110874,62.0,8.976894,6,5,1999,2000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1456,3.044522,85.0,9.486152,6,6,1978,1988,4.787492,6.673298,5.099866,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1457,4.262680,66.0,9.109746,7,9,1941,2006,0.000000,5.620401,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1458,3.044522,68.0,9.181735,5,6,1950,1996,0.000000,3.912023,6.937314,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0


In [180]:
total_ohe.iloc[1460:,:]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1460,3.044522,80.0,9.360741,5,6,1961,1961,0.000000,6.150603,4.976734,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1461,3.044522,81.0,9.565775,6,6,1958,1958,4.691348,6.828712,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1462,4.110874,74.0,9.534668,5,5,1997,1998,0.000000,6.674561,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1463,4.110874,78.0,9.208238,6,6,1998,1998,3.044522,6.401917,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1464,4.795791,43.0,8.518392,8,5,1992,1992,0.000000,5.575949,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,5.081404,21.0,7.568896,4,7,1970,1970,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2915,5.081404,21.0,7.546974,4,5,1970,1970,0.000000,5.533389,0.000000,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
2916,3.044522,160.0,9.903538,5,7,1960,1996,0.000000,7.110696,0.000000,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
2917,4.454347,62.0,9.253591,5,5,1992,1992,0.000000,5.823046,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0


In [181]:
house_df_org

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [182]:
house_train_data = total_ohe.iloc[:1460,:]

In [183]:
house_test_data = total_ohe.iloc[1460:,:]

In [184]:
house_test_data.reset_index(drop=True, inplace = True)

In [185]:
house_train_data

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.110874,65.0,9.042040,7,5,2003,2003,5.283204,6.561031,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1,3.044522,80.0,9.169623,6,8,1976,1976,0.000000,6.886532,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2,4.110874,68.0,9.328212,7,5,2001,2002,5.093750,6.188264,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
3,4.262680,60.0,9.164401,7,5,1915,1970,0.000000,5.379897,0.000000,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
4,4.110874,84.0,9.565284,8,5,2000,2000,5.860786,6.486161,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,4.110874,62.0,8.976894,6,5,1999,2000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1456,3.044522,85.0,9.486152,6,6,1978,1988,4.787492,6.673298,5.099866,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1457,4.262680,66.0,9.109746,7,9,1941,2006,0.000000,5.620401,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1458,3.044522,68.0,9.181735,5,6,1950,1996,0.000000,3.912023,6.937314,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0


In [186]:
house_test_data

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,3.044522,80.0,9.360741,5,6,1961,1961,0.000000,6.150603,4.976734,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1,3.044522,81.0,9.565775,6,6,1958,1958,4.691348,6.828712,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
2,4.110874,74.0,9.534668,5,5,1997,1998,0.000000,6.674561,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
3,4.110874,78.0,9.208238,6,6,1998,1998,3.044522,6.401917,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
4,4.795791,43.0,8.518392,8,5,1992,1992,0.000000,5.575949,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,5.081404,21.0,7.568896,4,7,1970,1970,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0
1455,5.081404,21.0,7.546974,4,5,1970,1970,0.000000,5.533389,0.000000,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
1456,3.044522,160.0,9.903538,5,7,1960,1996,0.000000,7.110696,0.000000,...,0.0,0.0,0.0,1,0.693359,0.0,0.0,0.0,0,0.0
1457,4.454347,62.0,9.253591,5,5,1992,1992,0.000000,5.823046,0.000000,...,0.0,0.0,0.0,1,0.000000,0.0,0.0,0.0,1,0.0


In [187]:
house_target


0       12.247699
1       12.109016
2       12.317171
3       11.849405
4       12.429220
          ...    
1455    12.072547
1456    12.254868
1457    12.493133
1458    11.864469
1459    11.901590
Name: SalePrice, Length: 1460, dtype: float64

In [188]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=False)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [189]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [190]:
lr_reg = LinearRegression()
lasso_reg = Lasso(alpha=0.001)
ridge_reg = Ridge(alpha=8)
xgb_reg = XGBRegressor(n_estimators=1000, learning_rate=0.05, 
                       colsample_bytree=0.5, subsample=0.8)
lgbm_reg = LGBMRegressor(n_estimators=1000, learning_rate=0.05, num_leaves=4, 
                         subsample=0.6, colsample_bytree=0.4, reg_lambda=10, n_jobs=-1)

In [192]:
# get_stacking_base_datasets( )은 넘파이 ndarray를 인자로 사용하므로 DataFrame을 넘파이로 변환. 
X_train_n = house_train_data.values
X_test_n = house_test_data.values
y_train_n = house_target.values

# 각 개별 기반(Base)모델이 생성한 학습용/테스트용 데이터 반환. 
ridge_train, ridge_test = get_stacking_base_datasets(ridge_reg, X_train_n, y_train_n, X_test_n, 5)
lasso_train, lasso_test = get_stacking_base_datasets(lasso_reg, X_train_n, y_train_n, X_test_n, 5)
xgb_train, xgb_test = get_stacking_base_datasets(xgb_reg, X_train_n, y_train_n, X_test_n, 5)  
lgbm_train, lgbm_test = get_stacking_base_datasets(lgbm_reg, X_train_n, y_train_n, X_test_n, 5)

Ridge  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
Lasso  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
XGBRegressor  model 시작 
	 폴드 세트:  0  시작 
[05:09:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  1  시작 
[05:09:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  2  시작 
[05:09:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  3  시작 
[05:09:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
	 폴드 세트:  4  시작 
[05:09:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
LGBMRegressor  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  

In [193]:
# 개별 모델이 반환한 학습 및 테스트용 데이터 세트를 Stacking 형태로 결합.  
Stack_final_X_train = np.concatenate((ridge_train, lasso_train, 
                                      xgb_train, lgbm_train), axis=1)
Stack_final_X_test = np.concatenate((ridge_test, lasso_test, 
                                     xgb_test, lgbm_test), axis=1)

# 최종 메타 모델은 라쏘 모델을 적용. 
meta_model_lasso = Lasso(alpha=0.0005)

#기반 모델의 예측값을 기반으로 새롭게 만들어진 학습 및 테스트용 데이터로 예측하고 RMSE 측정.
meta_model_lasso.fit(Stack_final_X_train, house_target)
final = meta_model_lasso.predict(Stack_final_X_test)
#mse = mean_squared_error(y_test , final)
#rmse = np.sqrt(mse)
#print('스태킹 회귀 모델의 최종 RMSE 값은:', rmse)

In [194]:
final

array([11.70624236, 11.98260637, 12.13005101, ..., 12.00996498,
       11.63413605, 12.31944304])

In [195]:
np.expm1(final)

array([121325.72127357, 159947.37276728, 185358.23002307, ...,
       164383.7480289 , 112885.26101827, 224008.34336116])

In [196]:
submit_house = pd.DataFrame()

In [197]:
submit_house['Id'] = np.arange(1461, 2920)

In [198]:
submit_house['SalePrice'] = np.expm1(final)

In [199]:
submit_house

,Id,SalePrice
0,1461,121325.721274
1,1462,159947.372767
2,1463,185358.230023
3,1464,195020.310962
4,1465,187675.438873
...,...,...
1454,2915,80630.617809
1455,2916,80537.628508
1456,2917,164383.748029
1457,2918,112885.261018


In [202]:
submit_house.round().to_csv('submit_house8.csv', index= False)

In [ ]:
submit_house.to_csv('submit_house3.csv', index= False)

In [ ]:
from sklearn.neural_network import MLPRegressor


In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(10,10,), activation='relu', \
                    alpha=0.01,
                    max_iter=5000)  # 객체 생성
 
mlp_model = mlp.fit(X_train_n, y_train_n)    # 훈련하기
mlp_final = mlp_model.predict(X_test_n)

In [ ]:
mlp_final

In [ ]:
X_train_n

In [ ]:
np.expm1(mlp_final)

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(10,10,), activation='relu', \
                    alpha=0.01,
                    max_iter=5000)  # 객체 생성
 
mlp_model = mlp.fit(X_train_n, y_train_n)    # 훈련하기
mlp_final = mlp_model.predict(X_test_n)

In [ ]:
def suhouse(asw):
  submit_house1 = pd.DataFrame()
  submit_house1['Id'] = np.arange(1461, 2920)
  submit_house1['SalePrice'] = asw
  submit_house1.to_csv('submit_house7.csv', index= False)


In [ ]:
suhouse(np.expm1(mlp_final))

In [ ]:
from sklearn.model_selection import GridSearchCV
mlp = MLPRegressor()
param_grid1 = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,1), (4, 8, 16, 8, 4) ,()],
          'activation': ['relu','tanh','logistic'],
          'alpha': [0.01, 0.1, 0.001, 0.0001, 0.05],
          'learning_rate': ['constant','adaptive'],
          'solver': ['adam', 'lbfgs', 'sgd']}

grid = GridSearchCV(estimator=mlp, param_grid = param_grid1, cv = 4, n_jobs=-1)
mlp_full_data = grid.fit(Stack_final_X_train, y_train_n)

In [ ]:
mlp_full_data.predict(Stack_final_X_test)

In [ ]:
mlp_full_data.predict(Stack_final_X_test)

In [ ]:
house_sumbmit1 = np.expm1(mlp_full_data.predict(Stack_final_X_test))

In [ ]:
suhouse(house_sumbmit1)

In [ ]:
mlp_full_data1 = grid.fit(house_train_data, house_target)

In [ ]:
np.expm1(mlp_full_data1.predict(house_test_data))

array([129227.57320193, 147214.35013726, 194876.96182976, ...,
       223081.87277426, 116592.33776385, 242232.28389924])

In [ ]:
suhouse(np.expm1(mlp_full_data1.predict(house_test_data)))

In [ ]:
mlp_full_data1.score(house_train_data, house_target)

0.8118439941162688

In [ ]:
mlp_full_data.score(house_train_data, house_target)

0.8118439941162688

In [ ]:
mlp_model.score(house_train_data, house_target)

0.7051294115666424